# Project "F#&* the world!" 

> Let's just say we have objective on 

llama llm fine tuning (the gguf file input is going to be "./Phi-3-mini-4k-instruct-q4.gguf"
and the output ""./Phi-3-mini-4k-instruct-q4_finetuned.gguf") WITHOUT Nvidia and WITH ARM cpu, WITH QLORA 4bit quantization then able to merge the model (NO Nvidia OR CUDA Specific CODE! NO ENVIRONMENT SPECIFIC PRODUCT LOCKIN), while having Limited memory budget (50% or 40% from total Memory).

### For first experiment, let's use pre-existing gguf 

> We're attempting to fine tuning QLora and VLLM without Nvidia, AMD, Intel and more portable device like ARM based processor or RISCV

In [1]:
%pip install torch transformers peft datasets llama-cpp-python

UsageError: Line magic function `%wget` not found.


#### Phi-3 Is our guinea pig model, We download it through wget

In [8]:
!wget "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf?download=true" -O "./Phi-3-mini-4k-instruct-q4.gguf"

--2024-12-14 07:19:23--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf?download=true
Resolving huggingface.co (huggingface.co)... 108.138.141.78, 108.138.141.7, 108.138.141.9, ...
Connecting to huggingface.co (huggingface.co)|108.138.141.78|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/8a83c7fb9049a9b2e92266fa7ad04933bb53aa1e85136b7b30f1b8000ff2edef?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-q4.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-q4.gguf%22%3B&Expires=1734394764&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDM5NDc2NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvOGE4M2M3ZmI5MDQ5YTliMmU

#### Iteration 0 : Fail ❌
> Governor : Claude 3.5 Sonnet

In [10]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32

class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features
        
        weight = linear.weight.data
        self.register_buffer('weight', self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer('bias', linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()

    def _extract_config(self):
    # Extract model configuration from GGUF using methods
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': self.llm.n_head(),  # Corrected: use n_head property
            'num_hidden_layers': self.llm.n_layer(),  # Corrected: use n_layer property
            'intermediate_size': self.llm.n_embd() * 4,
            'max_position_embeddings': self.llm.n_ctx(),
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)
        
        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False)
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False),
            'down_proj': torch.nn.Linear(intermediate_size, hidden_size, bias=False),
            'up_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False)
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
        # Map GGUF weight names to our architecture
        mapping = {
            'attention.wq': 'self_attn.q_proj',
            'attention.wk': 'self_attn.k_proj',
            'attention.wv': 'self_attn.v_proj',
            'attention.wo': 'self_attn.o_proj',
            'feed_forward.w1': 'mlp.gate_proj',
            'feed_forward.w2': 'mlp.down_proj',
            'feed_forward.w3': 'mlp.up_proj'
        }
        
        for gguf_pattern, our_pattern in mapping.items():
            if gguf_pattern in name:
                layer_num = int(name.split('.')[1])
                module = self.layers[layer_num].get_submodule(our_pattern)
                module.weight.copy_(tensor)
                break

    def forward(self, input_ids, attention_mask=None):
        hidden_states = self.embed_tokens(input_ids)
        
        for layer in self.layers:
            hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
            hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
            
        logits = self.lm_head(hidden_states)
        return logits

    def _attention_forward(self, attn, hidden_states, attention_mask=None):
        q = attn['q_proj'](hidden_states)
        k = attn['k_proj'](hidden_states)
        v = attn['v_proj'](hidden_states)
        
        # Simple scaled dot-product attention
        scale = 1.0 / math.sqrt(self.config['hidden_size'] // self.config['num_attention_heads'])
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale
        
        if attention_mask is not None:
            attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
        
        attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
        context = torch.matmul(attention_probs, v)
        
        return attn['o_proj'](context)

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)

# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.gguf"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=8,
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Apply custom quantization
model = quantize_model(model)

# LoRA setup
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": tokenized,
        "attention_mask": [[1] * len(tokens) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save back to GGUF
def save_to_gguf(merged_model, llm, output_path):
    """Save the merged model back to GGUF format."""
    with torch.no_grad():
        state_dict = llm.model.state_dict()
        
        # Copy weights from merged model to GGUF state dict
        for name, param in merged_model.named_parameters():
            if name in state_dict:
                # Ensure proper quantization
                quantized_param = CustomQuantLinear.quantize(
                    param.float(), bits=4, group_size=128
                )
                state_dict[name] = quantized_param.numpy()

        llm.save(output_path)

# Save the merged and quantized model
save_to_gguf(merged_model, llm, OUTPUT_PATH)

# Cleanup
del model
del merged_model

llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_

AttributeError: 'Llama' object has no attribute 'n_head'

#### Iteration 1 : Fail ❌
> Governor : Gemini 2.0

In [12]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32

class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features
        
        weight = linear.weight.data
        self.register_buffer('weight', self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer('bias', linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math

# ... (CustomQuantConfig and CustomQuantLinear classes remain the same)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()

    def _extract_config(self):
        # Extract model configuration from GGUF using methods
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': self.llm._model.n_head(),
            'num_hidden_layers': self.llm._model.n_layer(),
            'intermediate_size': self.llm.n_embd() * 4,
            'max_position_embeddings': self.llm.n_ctx(),
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)
        
        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False)
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False),
            'down_proj': torch.nn.Linear(intermediate_size, hidden_size, bias=False),
            'up_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False)
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
        # Map GGUF weight names to our architecture
        mapping = {
            'attention.wq': 'self_attn.q_proj',
            'attention.wk': 'self_attn.k_proj',
            'attention.wv': 'self_attn.v_proj',
            'attention.wo': 'self_attn.o_proj',
            'feed_forward.w1': 'mlp.gate_proj',
            'feed_forward.w2': 'mlp.down_proj',
            'feed_forward.w3': 'mlp.up_proj'
        }
        
        for gguf_pattern, our_pattern in mapping.items():
            if gguf_pattern in name:
                layer_num = int(name.split('.')[1])
                module = self.layers[layer_num].get_submodule(our_pattern)
                module.weight.copy_(tensor)
                break

    def forward(self, input_ids, attention_mask=None):
        hidden_states = self.embed_tokens(input_ids)
        
        for layer in self.layers:
            hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
            hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
            
        logits = self.lm_head(hidden_states)
        return logits

    def _attention_forward(self, attn, hidden_states, attention_mask=None):
        q = attn['q_proj'](hidden_states)
        k = attn['k_proj'](hidden_states)
        v = attn['v_proj'](hidden_states)
        
        # Simple scaled dot-product attention
        scale = 1.0 / math.sqrt(self.config['hidden_size'] // self.config['num_attention_heads'])
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale
        
        if attention_mask is not None:
            attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
        
        attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
        context = torch.matmul(attention_probs, v)
        
        return attn['o_proj'](context)

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)

# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.gguf"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=8,
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Apply custom quantization
model = quantize_model(model)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": tokenized,
        "attention_mask": [[1] * len(tokens) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save back to GGUF
def save_to_gguf(merged_model, llm, output_path):
    """Save the merged model back to GGUF format."""
    with torch.no_grad():
        state_dict = llm.model.state_dict()
        
        # Copy weights from merged model to GGUF state dict
        for name, param in merged_model.named_parameters():
            if name in state_dict:
                # Ensure proper quantization
                quantized_param = CustomQuantLinear.quantize(
                    param.float(), bits=4, group_size=128
                )
                state_dict[name] = quantized_param.numpy()

        llm.save(output_path)

# Save the merged and quantized model
save_to_gguf(merged_model, llm, OUTPUT_PATH)

# Cleanup
del model
del merged_model

llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10915 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_

AttributeError: 'LlamaModel' object has no attribute 'n_head'

#### Iteration 2: FAIL ❌
> Governor : Gemini 2.0


In [16]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math


@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32


class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()

    def _extract_config(self):
        # Extract model configuration from GGUF using methods
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': self.llm.n_head(),
            'num_hidden_layers': self.llm.n_layer(),
            'intermediate_size': self.llm.n_embd() * 4,
            'max_position_embeddings': self.llm.n_ctx(),
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)

        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False)
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False),
            'down_proj': torch.nn.Linear(intermediate_size, hidden_size, bias=False),
            'up_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False)
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
      # Map GGUF weight names to our architecture
      mapping = {
          'attention.wq': 'self_attn.q_proj',
          'attention.wk': 'self_attn.k_proj',
          'attention.wv': 'self_attn.v_proj',
          'attention.wo': 'self_attn.o_proj',
          'feed_forward.w1': 'mlp.gate_proj',
          'feed_forward.w2': 'mlp.down_proj',
          'feed_forward.w3': 'mlp.up_proj'
      }

      for gguf_pattern, our_pattern in mapping.items():
          if gguf_pattern in name:
            layer_num = int(name.split('.')[1])
            module = self.layers[layer_num].get_submodule(our_pattern)
            module.weight.copy_(tensor)
            break
            
    def forward(self, input_ids, attention_mask=None):
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
    def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(self.config['hidden_size'] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'](context)

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)

# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=8,
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": [tokens + [llm.token_eos()] for tokens in tokenized],
        "attention_mask": [[1] * (len(tokens) + 1) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save model
torch.save(merged_model.state_dict(), OUTPUT_PATH)

# Cleanup
del model
del merged_model

llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10907 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_

AttributeError: 'Llama' object has no attribute 'n_head'

#### Iteration 3 : Semi-Fail 🤔 (Inference able to Load, but Memory Consumption is Outrageous)
> Governor : Claude 3.5 Sonnet

In [ ]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math


@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32


class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()

    def _extract_config(self):
        # Extract model configuration from GGUF metadata
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': 32,  # From 'phi3.attention.head_count': '32'
            'num_hidden_layers': 32,    # From 'phi3.block_count': '32'
            'intermediate_size': 8192,   # From 'phi3.feed_forward_length': '8192'
            'max_position_embeddings': 4096,  # From 'phi3.context_length': '4096'
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)
        
        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False)
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False),
            'down_proj': torch.nn.Linear(intermediate_size, hidden_size, bias=False),
            'up_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False)
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
      # Map GGUF weight names to our architecture
      mapping = {
          'attention.wq': 'self_attn.q_proj',
          'attention.wk': 'self_attn.k_proj',
          'attention.wv': 'self_attn.v_proj',
          'attention.wo': 'self_attn.o_proj',
          'feed_forward.w1': 'mlp.gate_proj',
          'feed_forward.w2': 'mlp.down_proj',
          'feed_forward.w3': 'mlp.up_proj'
      }

      for gguf_pattern, our_pattern in mapping.items():
          if gguf_pattern in name:
            layer_num = int(name.split('.')[1])
            module = self.layers[layer_num].get_submodule(our_pattern)
            module.weight.copy_(tensor)
            break
            
    def forward(self, input_ids, attention_mask=None):
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
    def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(self.config['hidden_size'] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'](context)

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)

# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=8,
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": [tokens + [llm.token_eos()] for tokens in tokenized],
        "attention_mask": [[1] * (len(tokens) + 1) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save model
torch.save(merged_model.state_dict(), OUTPUT_PATH)

# Cleanup
del model
del merged_model

llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10907 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_

#### Iteration 4 : Fail ❌ (Biased towards CUDA Nvidia Only Environment!)
> Governor : Claude 3.5 Sonnet

In [ ]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math
import psutil
import gc

class MemoryManager:
    def __init__(self, memory_fraction=0.5):
        self.memory_fraction = memory_fraction
        self.total_memory = psutil.virtual_memory().total
        self.allocatable_memory = int(self.total_memory * self.memory_fraction)
        
    def get_optimal_batch_size(self, sample_input_size, model_size):
        """Calculate optimal batch size based on available memory"""
        memory_per_sample = sample_input_size * 4  # Rough estimate for float32
        memory_overhead = model_size * 4  # Model parameters
        available_memory = self.allocatable_memory - memory_overhead
        max_batch_size = max(1, available_memory // (memory_per_sample * 3))  # Factor of 3 for gradients and optimizer states
        return max_batch_size
    
    def get_optimal_chunk_size(self, sequence_length, hidden_size):
        """Calculate optimal chunk size for attention computation"""
        memory_per_token = hidden_size * 4  # Rough estimate for attention
        max_tokens = self.allocatable_memory // (memory_per_token * 3)
        return min(sequence_length, max(1, max_tokens // hidden_size))

    def check_memory(self):
        """Print current memory usage"""
        current_memory = psutil.Process().memory_info().rss
        print(f"Current memory usage: {current_memory / 1024 / 1024:.2f} MB")
        print(f"Allocatable memory: {self.allocatable_memory / 1024 / 1024:.2f} MB")
        return current_memory < self.allocatable_memory

class ChunkedAttention(torch.nn.Module):
    def __init__(self, hidden_size, num_attention_heads, memory_manager):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads
        self.memory_manager = memory_manager
        
    def forward(self, q, k, v, attention_mask=None):
        batch_size, seq_length, _ = q.size()
        chunk_size = self.memory_manager.get_optimal_chunk_size(seq_length, self.hidden_size)
        
        # Reshape for multi-head attention
        q = q.view(batch_size, seq_length, self.num_attention_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_length, self.num_attention_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_length, self.num_attention_heads, self.head_dim).transpose(1, 2)
        
        # Initialize output tensor
        attention_output = torch.zeros_like(v)
        
        # Process attention in chunks
        for i in range(0, seq_length, chunk_size):
            chunk_end = min(i + chunk_size, seq_length)
            
            # Calculate attention scores for current chunk
            attention_scores = torch.matmul(q[:, :, i:chunk_end], k.transpose(-2, -1))
            attention_scores = attention_scores / math.sqrt(self.head_dim)
            
            if attention_mask is not None:
                attention_scores = attention_scores.masked_fill(
                    attention_mask[:, None, i:chunk_end] == 0, 
                    float('-inf')
                )
            
            # Apply softmax and compute context
            attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
            context = torch.matmul(attention_probs, v)
            attention_output[:, :, i:chunk_end] = context
            
            # Clear cache
            del attention_scores, attention_probs, context
            torch.cuda.empty_cache() if torch.cuda.is_available() else gc.collect()
        
        # Reshape output
        attention_output = attention_output.transpose(1, 2).contiguous()
        return attention_output.view(batch_size, seq_length, self.hidden_size)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm, memory_manager):
        super().__init__()
        self.llm = llm
        self.memory_manager = memory_manager
        self.config = self._extract_config()
        self.setup_model_architecture()

    def _extract_config(self):
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': 32,  # From metadata
            'num_hidden_layers': 32,    # From metadata
            'intermediate_size': 8192,   # From metadata
            'max_position_embeddings': 4096,  # From metadata
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'chunked_attention': ChunkedAttention(
                hidden_size, 
                self.config['num_attention_heads'],
                self.memory_manager
            )
        })

    # ... (rest of the model implementation remains the same) ...

    def _attention_forward(self, attn, hidden_states, attention_mask=None):
        q = attn['q_proj'](hidden_states)
        k = attn['k_proj'](hidden_states)
        v = attn['v_proj'](hidden_states)
        
        # Use chunked attention
        context = attn['chunked_attention'](q, k, v, attention_mask)
        return attn['o_proj'](context)

# Memory manager initialization
memory_manager = MemoryManager(memory_fraction=0.5)  # Use 50% of system memory

# Load GGUF model with memory constraints
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=8,
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF with memory management
model = GGUFModel(llm, memory_manager)

# Calculate optimal batch size for training
sample_input = torch.zeros(1, 512, model.config['hidden_size'])  # Example input
optimal_batch_size = memory_manager.get_optimal_batch_size(
    sample_input.numel(),
    sum(p.numel() for p in model.parameters())
)

# Training setup with dynamic batch size
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=optimal_batch_size,
    gradient_accumulation_steps=max(1, 16 // optimal_batch_size),  # Adjust accumulation to maintain effective batch size
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

# Function to check memory during training
def training_callback(trainer, args, state, control, **kwargs):
    if not memory_manager.check_memory():
        print("Memory limit exceeded, reducing batch size")
        trainer.args.per_device_train_batch_size = max(1, trainer.args.per_device_train_batch_size // 2)
        trainer.args.gradient_accumulation_steps *= 2
    return control

# Add callback to trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    callbacks=[training_callback]
)

# Train with memory monitoring
try:
    trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e):
        print("OOM detected, trying to recover...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        # Reduce batch size and try again
        trainer.args.per_device_train_batch_size //= 2
        trainer.args.gradient_accumulation_steps *= 2
        trainer.train()

#### Iteration 5 : Semi-Fail 🤔 (Inference able to Load, but Memory Consumption is Outrageous)
> Governor : Google Gemini 2.0

> THE FUCK IS THAT?!, NO FUCKING CUDA CODE WITH NO EXCEPTION EVEN FOR CHECKING. THAT'S GOING TO BE IMPLEMENTED LATER

In [5]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math
import os
import gc # garbage collector

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32


class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()

    def _extract_config(self):
        # Extract model configuration from GGUF metadata
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': 32,  # From 'phi3.attention.head_count': '32'
            'num_hidden_layers': 32,    # From 'phi3.block_count': '32'
            'intermediate_size': 8192,   # From 'phi3.feed_forward_length': '8192'
            'max_position_embeddings': 4096,  # From 'phi3.context_length': '4096'
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)
        
        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False)
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False),
            'down_proj': torch.nn.Linear(intermediate_size, hidden_size, bias=False),
            'up_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False)
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
      # Map GGUF weight names to our architecture
      mapping = {
          'attention.wq': 'self_attn.q_proj',
          'attention.wk': 'self_attn.k_proj',
          'attention.wv': 'self_attn.v_proj',
          'attention.wo': 'self_attn.o_proj',
          'feed_forward.w1': 'mlp.gate_proj',
          'feed_forward.w2': 'mlp.down_proj',
          'feed_forward.w3': 'mlp.up_proj'
      }

      for gguf_pattern, our_pattern in mapping.items():
          if gguf_pattern in name:
            layer_num = int(name.split('.')[1])
            module = self.layers[layer_num.get_submodule(our_pattern)]
            module.weight.copy_(tensor)
            break
            
    def forward(self, input_ids, attention_mask=None):
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
    def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(self.config['hidden_size'] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'](context)

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'(intermediate)]


# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=os.cpu_count(),
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": [tokens + [llm.token_eos()] for tokens in tokenized],
        "attention_mask": [[1] * (len(tokens) + 1) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1, # Smaller batch size
    gradient_accumulation_steps=2,  # Accumulate gradients before updating
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save model
torch.save(merged_model.state_dict(), OUTPUT_PATH)

# Explicit memory management
del model
del merged_model

gc.collect()
torch.cuda.empty_cache() # To release CUDA memory (it's useless now but this line will prevent an error if it was a CUDA implementation)
print("Memory cleanup completed.")

<>:172: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:172: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
/Users/albertstarfield/Documents/misc/AI/project-zephyrine/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/hr/96cc5bsn3f52rnk149qdnc5h0000gs/T/ipykernel_23625/1632802455.py:172: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  return mlp['down_proj'(intermediate)]
llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - k

KeyboardInterrupt: 

#### Iteration 6 : Semi-Fail 🤔 (Inference able to Load, but Memory Consumption is Outrageous)
> Governor : Google Gemini 2.0

In [ ]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math
import os
import gc
import psutil  # For system memory monitoring
import time   # For timing responsiveness

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32


class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()
        self.memory_responsiveness = 1.0 # initial responsiveness
        self.last_memory_check = time.time()
        self.attention_limit_factor = 1.0 #initial attention limit
        self.memory_check_interval = 5 # check memory every 5 seconds
    
    def _extract_config(self):
        # Extract model configuration from GGUF metadata
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': 32,  # From 'phi3.attention.head_count': '32'
            'num_hidden_layers': 32,    # From 'phi3.block_count': '32'
            'intermediate_size': 8192,   # From 'phi3.feed_forward_length': '8192'
            'max_position_embeddings': 4096,  # From 'phi3.context_length': '4096'
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = torch.nn.Embedding(config['vocab_size'], config['hidden_size'])
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False)
        
        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'k_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'v_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False),
            'o_proj': torch.nn.Linear(hidden_size, hidden_size, bias=False)
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False),
            'down_proj': torch.nn.Linear(intermediate_size, hidden_size, bias=False),
            'up_proj': torch.nn.Linear(hidden_size, intermediate_size, bias=False)
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
      # Map GGUF weight names to our architecture
      mapping = {
          'attention.wq': 'self_attn.q_proj',
          'attention.wk': 'self_attn.k_proj',
          'attention.wv': 'self_attn.v_proj',
          'attention.wo': 'self_attn.o_proj',
          'feed_forward.w1': 'mlp.gate_proj',
          'feed_forward.w2': 'mlp.down_proj',
          'feed_forward.w3': 'mlp.up_proj'
      }

      for gguf_pattern, our_pattern in mapping.items():
          if gguf_pattern in name:
            layer_num = int(name.split('.')[1])
            module = self.layers[layer_num].get_submodule(our_pattern)
            module.weight.copy_(tensor)
            break
    def _check_memory_and_responsiveness(self):
        """Monitors memory and responsiveness, adjusting attention limit factor."""
        
        current_time = time.time()
        if current_time - self.last_memory_check < self.memory_check_interval:
            return # Only check memory every so often

        self.last_memory_check = current_time
        
        # Calculate Memory Usage
        memory_usage = psutil.virtual_memory().percent
        # Basic responsiveness check (can be improved)
        start_time = time.time()
        _ = [i for i in range(1000)] # Simple calculation
        end_time = time.time()
        responsiveness = (end_time-start_time)*1000 # in ms
        
        # adjust memory responsiveness
        if responsiveness > 10:
            self.memory_responsiveness *= 0.9
        else:
            self.memory_responsiveness *= 1.1
            self.memory_responsiveness = min(self.memory_responsiveness,1.0)

        # Adjust attention limit factor
        if memory_usage > 80 or self.memory_responsiveness<0.5:
            self.attention_limit_factor = max(0.2, self.memory_responsiveness)
            print(f"Memory Usage High ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
        elif memory_usage > 70:
            self.attention_limit_factor = max(0.4, self.memory_responsiveness)
            print(f"Memory Usage Medium ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
        else:
            self.attention_limit_factor = 1.0
            print(f"Memory usage low, responsiveness {self.memory_responsiveness}. Attention limit reset to 1.0")

    def forward(self, input_ids, attention_mask=None):
        self._check_memory_and_responsiveness() # Check before each forward pass
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
    def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Apply dynamic attention limiting:
      q = q[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      k = k[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      v = v[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(q.shape[-1] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'](context)

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)


# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=os.cpu_count(),
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": [tokens + [llm.token_eos()] for tokens in tokenized],
        "attention_mask": [([1] * (len(tokens) + 1)) for tokens in tokenized] # use a regular list comprehension syntax here
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save model
torch.save(merged_model.state_dict(), OUTPUT_PATH)

# Explicit memory management
del model
del merged_model

gc.collect()
print("Memory cleanup completed.")

/Users/albertstarfield/Documents/misc/AI/project-zephyrine/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              =

#### Iteration 7 : Semi-Fail 🤔 (Inference able to Load, but the fine tuning part...)
> Governor : Google Gemini 2.0

> Changes on to make sure that all part is processed in 4bit Accuracy?


In [3]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math
import os
import gc
import psutil  # For system memory monitoring
import time   # For timing responsiveness

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32


class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()
        self.memory_responsiveness = 1.0 # initial responsiveness
        self.last_memory_check = time.time()
        self.attention_limit_factor = 1.0 #initial attention limit
        self.memory_check_interval = 5 # check memory every 5 seconds
    
    def _extract_config(self):
        # Extract model configuration from GGUF metadata
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': 32,  # From 'phi3.attention.head_count': '32'
            'num_hidden_layers': 32,    # From 'phi3.block_count': '32'
            'intermediate_size': 8192,   # From 'phi3.feed_forward_length': '8192'
            'max_position_embeddings': 4096,  # From 'phi3.context_length': '4096'
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = CustomQuantLinear(torch.nn.Embedding(config['vocab_size'], config['hidden_size']))
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = CustomQuantLinear(torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False))
        
        # Load weights from GGUF
        self._load_weights_from_gguf()

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
            'k_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
            'v_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
            'o_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False))
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, intermediate_size, bias=False)),
            'down_proj': CustomQuantLinear(torch.nn.Linear(intermediate_size, hidden_size, bias=False)),
            'up_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, intermediate_size, bias=False))
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
      # Map GGUF weight names to our architecture
      mapping = {
          'attention.wq': 'self_attn.q_proj',
          'attention.wk': 'self_attn.k_proj',
          'attention.wv': 'self_attn.v_proj',
          'attention.wo': 'self_attn.o_proj',
          'feed_forward.w1': 'mlp.gate_proj',
          'feed_forward.w2': 'mlp.down_proj',
          'feed_forward.w3': 'mlp.up_proj'
      }

      for gguf_pattern, our_pattern in mapping.items():
          if gguf_pattern in name:
            layer_num = int(name.split('.')[1])
            module = self.layers[layer_num].get_submodule(our_pattern)
            module.weight.copy_(tensor)
            break
    def _check_memory_and_responsiveness(self):
        """Monitors memory and responsiveness, adjusting attention limit factor."""
        
        current_time = time.time()
        if current_time - self.last_memory_check < self.memory_check_interval:
            return # Only check memory every so often

        self.last_memory_check = current_time
        
        # Calculate Memory Usage
        memory_usage = psutil.virtual_memory().percent
        # Basic responsiveness check (can be improved)
        start_time = time.time()
        _ = [i for i in range(1000)] # Simple calculation
        end_time = time.time()
        responsiveness = (end_time-start_time)*1000 # in ms
        
        # adjust memory responsiveness
        if responsiveness > 10:
            self.memory_responsiveness *= 0.9
        else:
            self.memory_responsiveness *= 1.1
            self.memory_responsiveness = min(self.memory_responsiveness,1.0)

        # Adjust attention limit factor
        if memory_usage > 80 or self.memory_responsiveness<0.5:
            self.attention_limit_factor = max(0.2, self.memory_responsiveness)
            print(f"Memory Usage High ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
        elif memory_usage > 70:
            self.attention_limit_factor = max(0.4, self.memory_responsiveness)
            print(f"Memory Usage Medium ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
        else:
            self.attention_limit_factor = 1.0
            print(f"Memory usage low, responsiveness {self.memory_responsiveness}. Attention limit reset to 1.0")

    def forward(self, input_ids, attention_mask=None):
        self._check_memory_and_responsiveness() # Check before each forward pass
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
    def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Apply dynamic attention limiting:
      q = q[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      k = k[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      v = v[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(q.shape[-1] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'(context)]

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)


# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=os.cpu_count(),
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": [tokens + [llm.token_eos()] for tokens in tokenized],
        "attention_mask": [([1] * (len(tokens) + 1)) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save model
torch.save(merged_model.state_dict(), OUTPUT_PATH)

# Explicit memory management
del model
del merged_model

gc.collect()
print("Memory cleanup completed.")

<>:212: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:212: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
/var/folders/hr/96cc5bsn3f52rnk149qdnc5h0000gs/T/ipykernel_38845/2136757797.py:212: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  return attn['o_proj'(context)]
llama_load_model_from_file: using device Metal (Apple M2 Pro) - 10916 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:         

AttributeError: 'Embedding' object has no attribute 'in_features'

#### Iteration 8 : Semi-Fail 🤔 (I Don't know anymore, probably llama_cpp issue?)
> Governor : Google Gemini 2.0

In [14]:
import torch
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from llama_cpp import Llama
import numpy as np
from typing import Dict, Any
from dataclasses import dataclass
import math
import os
import gc
import psutil  # For system memory monitoring
import time   # For timing responsiveness

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32


class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

class GGUFModel(torch.nn.Module):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
        self.config = self._extract_config()
        self.setup_model_architecture()
        self.memory_responsiveness = 1.0 # initial responsiveness
        self.last_memory_check = time.time()
        self.attention_limit_factor = 1.0 #initial attention limit
        self.memory_check_interval = 5 # check memory every 5 seconds
    
    def _extract_config(self):
        # Extract model configuration from GGUF metadata
        return {
            'vocab_size': self.llm.n_vocab(),
            'hidden_size': self.llm.n_embd(),
            'num_attention_heads': 32,  # From 'phi3.attention.head_count': '32'
            'num_hidden_layers': 32,    # From 'phi3.block_count': '32'
            'intermediate_size': 8192,   # From 'phi3.feed_forward_length': '8192'
            'max_position_embeddings': 4096,  # From 'phi3.context_length': '4096'
        }

    def setup_model_architecture(self):
        config = self.config
        self.embed_tokens = CustomQuantLinear(torch.nn.Linear(config['vocab_size'], config['hidden_size'], bias=False)) #quantize a Linear layer first
        self.embed_tokens.weight = torch.nn.Parameter(self.embed_tokens.quantize(torch.randn(config['vocab_size'], config['hidden_size'])))
        self.layers = torch.nn.ModuleList([
            self._create_layer() for _ in range(config['num_hidden_layers'])
        ])
        self.lm_head = CustomQuantLinear(torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False))

    def _create_layer(self):
        config = self.config
        return torch.nn.ModuleDict({
            'self_attn': self._create_attention(),
            'mlp': self._create_mlp()
        })

    def _create_attention(self):
        hidden_size = self.config['hidden_size']
        return torch.nn.ModuleDict({
            'q_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
            'k_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
            'v_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
            'o_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False))
        })

    def _create_mlp(self):
        hidden_size = self.config['hidden_size']
        intermediate_size = self.config['intermediate_size']
        return torch.nn.ModuleDict({
            'gate_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, intermediate_size, bias=False)),
            'down_proj': CustomQuantLinear(torch.nn.Linear(intermediate_size, hidden_size, bias=False)),
            'up_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, intermediate_size, bias=False))
        })

    def _load_weights_from_gguf(self):
        with torch.no_grad():
            state_dict = self.llm.model.state_dict()
            for name, tensor in state_dict.items():
                if isinstance(tensor, (np.ndarray, torch.Tensor)):
                    tensor = torch.from_numpy(tensor) if isinstance(tensor, np.ndarray) else tensor
                    if 'embed' in name:
                        self.embed_tokens.weight.copy_(tensor)
                    elif 'lm_head' in name:
                        self.lm_head.weight.copy_(tensor)
                    else:
                        # Map other weights appropriately
                        self._map_layer_weights(name, tensor)

    def _map_layer_weights(self, name, tensor):
      # Map GGUF weight names to our architecture
      mapping = {
          'attention.wq': 'self_attn.q_proj',
          'attention.wk': 'self_attn.k_proj',
          'attention.wv': 'self_attn.v_proj',
          'attention.wo': 'self_attn.o_proj',
          'feed_forward.w1': 'mlp.gate_proj',
          'feed_forward.w2': 'mlp.down_proj',
          'feed_forward.w3': 'mlp.up_proj'
      }

      for gguf_pattern, our_pattern in mapping.items():
          if gguf_pattern in name:
            layer_num = int(name.split('.')[1])
            module = self.layers[layer_num].get_submodule(our_pattern)
            module.weight.copy_(tensor)
            break
    def _check_memory_and_responsiveness(self):
        """Monitors memory and responsiveness, adjusting attention limit factor."""
        
        current_time = time.time()
        if current_time - self.last_memory_check < self.memory_check_interval:
            return # Only check memory every so often

        self.last_memory_check = current_time
        
        # Calculate Memory Usage
        memory_usage = psutil.virtual_memory().percent
        # Basic responsiveness check (can be improved)
        start_time = time.time()
        _ = [i for i in range(1000)] # Simple calculation
        end_time = time.time()
        responsiveness = (end_time-start_time)*1000 # in ms
        
        # adjust memory responsiveness
        if responsiveness > 10:
            self.memory_responsiveness *= 0.9
        else:
            self.memory_responsiveness *= 1.1
            self.memory_responsiveness = min(self.memory_responsiveness,1.0)

        # Adjust attention limit factor
        if memory_usage > 80 or self.memory_responsiveness<0.5:
            self.attention_limit_factor = max(0.2, self.memory_responsiveness)
            print(f"Memory Usage High ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
        elif memory_usage > 70:
            self.attention_limit_factor = max(0.4, self.memory_responsiveness)
            print(f"Memory Usage Medium ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
        else:
            self.attention_limit_factor = 1.0
            print(f"Memory usage low, responsiveness {self.memory_responsiveness}. Attention limit reset to 1.0")

    def forward(self, input_ids, attention_mask=None):
        self._check_memory_and_responsiveness() # Check before each forward pass
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
    def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Apply dynamic attention limiting:
      q = q[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      k = k[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      v = v[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(q.shape[-1] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'(context)]

    def _mlp_forward(self, mlp, hidden_states):
        gate_output = mlp['gate_proj'](hidden_states)
        up_output = mlp['up_proj'](hidden_states)
        intermediate = gate_output * torch.nn.functional.gelu(up_output)
        return mlp['down_proj'](intermediate)


# Load GGUF model
INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

llm = Llama(
    model_path=INPUT_PATH,
    n_ctx=4096,
    n_threads=os.cpu_count(),
    vocab_only=False,
    use_mmap=True,
    use_mlock=False,
)

# Create model from GGUF
model = GGUFModel(llm)

# Inference test
test_prompt = "What is the capital of France?"
test_input_ids = [llm.tokenize(test_prompt.encode())]
test_attention_mask = [[1] * len(test_input_ids[0])]

with torch.no_grad():
    test_input_ids = torch.tensor(test_input_ids)
    test_attention_mask = torch.tensor(test_attention_mask)
    test_output = model(test_input_ids, attention_mask=test_attention_mask)
    
    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

print("Inference Test:")
print("Prompt:", test_prompt)
print("Response:", predicted_text)

# LoRA setup
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# Prepare dataset
dataset = load_dataset("text", data_files={"train": "my_data.txt"})

def tokenize_function(examples):
    # Use GGUF's tokenizer
    tokenized = [llm.tokenize(text.encode()) for text in examples["text"]]
    return {
        "input_ids": [tokens + [llm.token_eos()] for tokens in tokenized],
        "attention_mask": [([1] * (len(tokens) + 1)) for tokens in tokenized]
    }

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)
train_dataset = tokenized_dataset["train"]

# Training setup
training_args = TrainingArguments(
    output_dir="./phi3_mini_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    save_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    save_total_limit=2,
    max_grad_norm=None,
    warmup_ratio=0.03,
    fp16=False,
    bf16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train
trainer.train()

# Merge LoRA weights
merged_model = model.merge_and_unload()

# Save model
torch.save(merged_model.state_dict(), OUTPUT_PATH)

# Explicit memory management
del model
del merged_model

gc.collect()
print("Memory cleanup completed.")

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 256 (1566636207.py, line 268)

#### Iteration 9 : Semi-Fail 🤔 (I Don't know anymore, probably llama_cpp issue? I Gave up implementing llama_cpp)
> Governor : Google Gemini 2.0, Google Experimental 1206, Mixed with Claude 3.5 Sonnet for Proof Reading

> What changed is that we dropped support of llama_cpp and go MANUEL mode and drifting like a init-d pizza delivery driver. Power of manual/MANUEL [GAS GAS GAS](https://www.youtube.com/watch?v=UAIMoRuzps4)

In [20]:
import torch
import numpy as np
import os
import struct
import time
import psutil
import gc
from typing import Dict, Any
from dataclasses import dataclass
from torch.nn import functional as F
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import math
from peft import LoraConfig, get_peft_model  # Corrected import

# ----------------------- GGUF Parsing Utilities -----------------------
class GGUFParser:
    def __init__(self, model_path):
        self.model_path = model_path
        self.f = open(self.model_path, 'rb')
        self.magic = self.read_uint32()
        self.version = self.read_uint32()
        self.tensor_infos = self._parse_metadata()

    def read_uint32(self):
      return struct.unpack("<I", self.f.read(4))[0]

    def read_int32(self):
      return struct.unpack("<i", self.f.read(4))[0]

    def read_string(self):
        length = self.read_uint32()
        return self.f.read(length).decode()

    def read_bytes(self):
        length = self.read_uint32()
        return self.f.read(length)

    def read_tensor_data(self, tensor_info):
         self.f.seek(tensor_info['data_offset'])
         num_bytes = np.prod(tensor_info['shape']) * tensor_info['dtype_size']
         raw_bytes = self.f.read(num_bytes)
         data_np = np.frombuffer(raw_bytes, dtype=tensor_info['dtype']).reshape(tensor_info['shape'])
         return torch.from_numpy(data_np)
    
    def _parse_metadata(self):
        metadata = {}
        n_kv = self.read_uint32()

        for _ in range(n_kv):
            key = self.read_string()
            val_type = self.read_uint32()

            if val_type == 0:
                value = self.read_string()
            elif val_type == 1:
                value = self.read_uint32()
            elif val_type == 2:
                value = self.read_int32()
            elif val_type == 3:
                value = struct.unpack("<f", self.f.read(4))[0]
            elif val_type == 4:
                value = struct.unpack("<d", self.f.read(8))[0]
            elif val_type == 5:
               value = self.read_bytes()
            else:
                raise ValueError(f"Unknown value type {val_type}")
            metadata[key] = value

        n_tensors = self.read_uint32()
        tensor_infos = {}
        for _ in range(n_tensors):
            name = self.read_string()
            n_dims = self.read_uint32()
            shape = [self.read_uint32() for _ in range(n_dims)]
            dtype = self.read_uint32()
            data_offset = self.read_uint32()
            tensor_infos[name] = {
                'name':name,
                'shape': shape,
                'dtype': dtype,
                'data_offset': data_offset,
                'dtype_size': self.get_dtype_size(dtype)
            }
        return tensor_infos, metadata
    
    def get_dtype_size(self, dtype):
        if dtype == 0: return 4 #float32
        elif dtype == 1: return 2 #float16
        elif dtype == 2: return 4 #int32
        elif dtype == 3: return 1 #int8
        elif dtype == 4: return 1 #uint8
        elif dtype == 5: return 2 #int16
        elif dtype == 6: return 2 #bfloat16
        else: return 1
    
    def get_metadata(self):
      return self.tensor_infos[1]
    
    def get_tensor_info(self):
      return self.tensor_infos[0]

# ----------------------- Quantization Utilities -----------------------

@dataclass
class CustomQuantConfig:
    bits: int = 4
    group_size: int = 128
    dtype = torch.float32

class CustomQuantLinear(torch.nn.Module):
    def __init__(self, linear, bits=4, group_size=128):
        super().__init__()
        self.bits = bits
        self.group_size = group_size
        self.in_features = linear.in_features
        self.out_features = linear.out_features

        weight = linear.weight.data
        self.register_buffer("weight", self.quantize(weight))
        if linear.bias is not None:
            self.register_buffer("bias", linear.bias.data)
        else:
            self.bias = None

    def quantize(self, weight):
        orig_shape = weight.shape
        weight = weight.reshape(-1, self.group_size)
        weight_max = weight.max(dim=1, keepdim=True)[0]
        weight_min = weight.min(dim=1, keepdim=True)[0]
        scale = (weight_max - weight_min) / (2**self.bits - 1)
        zero = weight_min
        weight_q = torch.round((weight - zero) / scale) * scale + zero
        weight_q = weight_q.reshape(orig_shape)
        return weight_q

    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

# ----------------------- Model Definition -----------------------

class GGUFModel(torch.nn.Module):
  def __init__(self, model_path):
      super().__init__()
      self.gguf_parser = GGUFParser(model_path)
      self.tensor_info = self.gguf_parser.get_tensor_info()
      self.metadata = self.gguf_parser.get_metadata()
      self.config = self._extract_config()
      self._build_model()
      self.memory_responsiveness = 1.0 # initial responsiveness
      self.last_memory_check = time.time()
      self.attention_limit_factor = 1.0 #initial attention limit
      self.memory_check_interval = 5 # check memory every 5 seconds

  def _extract_config(self):
      return {
          'vocab_size': int(self.metadata.get('phi3.vocab_size')),
          'hidden_size': int(self.metadata.get('phi3.embedding_length')),
          'num_attention_heads': int(self.metadata.get('phi3.attention.head_count')),
          'num_hidden_layers': int(self.metadata.get('phi3.block_count')),
          'intermediate_size': int(self.metadata.get('phi3.feed_forward_length')),
          'max_position_embeddings': int(self.metadata.get('phi3.context_length')),
          'dtype': self.get_dtype(int(self.metadata.get('general.file_type', 0)))
      }
  
  def get_dtype(self, dtype):
    if dtype == 0: return torch.float32
    elif dtype == 1: return torch.float16
    elif dtype == 2: return torch.int32
    elif dtype == 3: return torch.int8
    elif dtype == 4: return torch.uint8
    elif dtype == 5: return torch.int16
    elif dtype == 6: return torch.bfloat16
    else: return torch.float32

  def _build_model(self):
      config = self.config
      self.embed_tokens = CustomQuantLinear(torch.nn.Linear(config['vocab_size'], config['hidden_size'], bias=False))
      self.layers = torch.nn.ModuleList([self._create_layer() for _ in range(config['num_hidden_layers'])])
      self.lm_head = CustomQuantLinear(torch.nn.Linear(config['hidden_size'], config['vocab_size'], bias=False))
        
  def _create_layer(self):
      return torch.nn.ModuleDict({
          'self_attn': self._create_attention(),
          'mlp': self._create_mlp()
      })
  
  def _create_attention(self):
      hidden_size = self.config['hidden_size']
      return torch.nn.ModuleDict({
          'q_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
          'k_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
          'v_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False)),
          'o_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, hidden_size, bias=False))
      })

  def _create_mlp(self):
      hidden_size = self.config['hidden_size']
      intermediate_size = self.config['intermediate_size']
      return torch.nn.ModuleDict({
          'gate_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, intermediate_size, bias=False)),
          'down_proj': CustomQuantLinear(torch.nn.Linear(intermediate_size, hidden_size, bias=False)),
          'up_proj': CustomQuantLinear(torch.nn.Linear(hidden_size, intermediate_size, bias=False))
      })

  def load_tensor_from_disk(self, name):
    tensor_info = self.tensor_info[name]
    return self.gguf_parser.read_tensor_data(tensor_info)

  def _check_memory_and_responsiveness(self):
      """Monitors memory and responsiveness, adjusting attention limit factor."""
      
      current_time = time.time()
      if current_time - self.last_memory_check < self.memory_check_interval:
          return # Only check memory every so often

      self.last_memory_check = current_time
      
      # Calculate Memory Usage
      memory_usage = psutil.virtual_memory().percent
      # Basic responsiveness check (can be improved)
      start_time = time.time()
      _ = [i for i in range(1000)] # Simple calculation
      end_time = time.time()
      responsiveness = (end_time-start_time)*1000 # in ms
      
      # adjust memory responsiveness
      if responsiveness > 10:
          self.memory_responsiveness *= 0.9
      else:
          self.memory_responsiveness *= 1.1
          self.memory_responsiveness = min(self.memory_responsiveness,1.0)

      # Adjust attention limit factor
      if memory_usage > 80 or self.memory_responsiveness<0.5:
          self.attention_limit_factor = max(0.2, self.memory_responsiveness)
          print(f"Memory Usage High ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
      elif memory_usage > 70:
          self.attention_limit_factor = max(0.4, self.memory_responsiveness)
          print(f"Memory Usage Medium ({memory_usage}%), responsiveness {self.memory_responsiveness}. Reduced attention limit to {self.attention_limit_factor:.2f}.")
      else:
          self.attention_limit_factor = 1.0
          print(f"Memory usage low, responsiveness {self.memory_responsiveness}. Attention limit reset to 1.0")


  def forward(self, input_ids, attention_mask=None):
        self._check_memory_and_responsiveness()
        hidden_states = self.embed_tokens(input_ids)
        print("Hidden states after embedding:", hidden_states.shape)

        for i, layer in enumerate(self.layers):
          print(f"Layer {i} input:", hidden_states.shape)
          hidden_states = self._attention_forward(layer['self_attn'], hidden_states, attention_mask)
          print(f"Layer {i} after attention:", hidden_states.shape)
          hidden_states = self._mlp_forward(layer['mlp'], hidden_states)
          print(f"Layer {i} after mlp:", hidden_states.shape)

        logits = self.lm_head(hidden_states)
        print("Logits shape:", logits.shape)
        return logits
    
  def _attention_forward(self, attn, hidden_states, attention_mask=None):
      q = attn['q_proj'](hidden_states)
      k = attn['k_proj'](hidden_states)
      v = attn['v_proj'](hidden_states)

      # Apply dynamic attention limiting:
      q = q[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      k = k[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]
      v = v[:, :, :int(self.config['hidden_size'] * self.attention_limit_factor)]

      # Simple scaled dot-product attention
      scale = 1.0 / math.sqrt(q.shape[-1] // self.config['num_attention_heads'])
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) * scale

      if attention_mask is not None:
          attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
      
      attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
      context = torch.matmul(attention_probs, v)
      
      return attn['o_proj'](context)
      
  def _mlp_forward(self, mlp, hidden_states):
      gate_output = mlp['gate_proj'](hidden_states)
      up_output = mlp['up_proj'](hidden_states)
      intermediate = gate_output * torch.nn.functional.gelu(up_output)
      return mlp['down_proj'](intermediate)

# ----------------------- Inference/Fine-tuning -----------------------
# ----------------------- Inference/Fine-tuning -----------------------
def main():
  INPUT_PATH = "./Phi-3-mini-4k-instruct-q4.gguf"
  OUTPUT_PATH = "./Phi-3-mini-4k-instruct-q4_finetuned.pth"

  # Load GGUF model
  model = GGUFModel(INPUT_PATH)

  #load model weights from disk
  with torch.no_grad():
    model.embed_tokens.weight = torch.nn.Parameter(model.load_tensor_from_disk("model.embed.weight"))
    model.lm_head.weight = torch.nn.Parameter(model.load_tensor_from_disk("model.lm_head.weight"))
    for i in range(len(model.layers)):
        for name, module in model.layers[i].items():  # Fixed syntax
            for k, sub_module in module.items():
                sub_module.weight = torch.nn.Parameter(model.load_tensor_from_disk(f"model.layers.{i}.{name}.{k}.weight"))

  # Inference test
  test_prompt = "What is the capital of France?"
  # Assuming the tokens are strings:
  test_input_ids = torch.tensor([[model.gguf_parser.metadata['tokenizer.ggml.tokens'].index(token) for token in test_prompt.split()]])
  test_attention_mask = torch.ones(test_input_ids.shape, dtype=torch.int32)

  with torch.no_grad():
    test_output = model(test_input_ids, test_attention_mask)

    predicted_ids = torch.argmax(test_output, dim=-1).squeeze()
    # You need to implement detokenization
    # predicted_text = llm.detokenize(predicted_ids.tolist()).decode()

  print("Inference Test:")
  print("Prompt:", test_prompt)
  # print("Response:", predicted_text)

if __name__ == "__main__":
  main()

ValueError: Unknown value type 24